# SQL Data Exploration 

For the purpose of keeping the SQL portion of this project available in our notebooks, we'll create an SQLite connection.

Our team at Pillow Palooza has exported the clean and preprocessed data and created a schema for us to use in our analysis. We have also included extra columns (from public data sources) to enhance the analysis. 

After meeting with your team, you wrote some questions to guide you along the analysis:
1. What are the most popular neighborhoods for short-term rentals in New York City?
2. What is the average rental price for short-term rentals in New York City, and how does it vary by neighborhood and property type?
3. What are the most commonly rented property types on Airbnb in New York City, and how does this vary by neighborhood?
4. What is the average length of stay for short-term rentals in New York City, and how does this vary by neighborhood and property type?
5. How has demand for short-term rentals in New York City changed over time, and are there any seasonal trends that could impact business decisions?

In [1]:
import pandas as pd
import sqlite3


In [2]:
# create the SQLite connection

cnn = sqlite3.connect('palooza.db')

In [3]:
# import the enriched data

prices = pd.read_csv('data_enriched/query_prices.csv')
room_types = pd.read_csv('data_enriched/query_room_types.csv')
reviews = pd.read_csv('data_enriched/query_reviews.csv')

In [4]:
# Save the data to SQLite

prices.to_sql('prices', cnn, if_exists='replace')
room_types.to_sql('room_types', cnn, if_exists='replace')
reviews.to_sql('reviews', cnn, if_exists='replace')



25209

In [5]:
# Load the ipython-sql extension
%load_ext sql

# Connect to the SQLite database
%sql sqlite:///palooza.db

'Connected: @palooza.db'

In [6]:
%%sql

SELECT COUNT(*)
FROM prices;


 * sqlite:///palooza.db
Done.


COUNT(*)
25202


In [7]:
%%sql

SELECT COUNT(*)
FROM room_types;

 * sqlite:///palooza.db
Done.


COUNT(*)
25209


In [8]:
%%sql

SELECT COUNT(*)
FROM reviews;

 * sqlite:///palooza.db
Done.


COUNT(*)
25209


## Questions 

1. What is the most common room type in NYC Airbnb listings?

In [9]:
%%sql
SELECT room_type, COUNT(*) type_count
FROM room_types
GROUP BY room_type
ORDER BY 2 DESC

 * sqlite:///palooza.db
Done.


room_type,type_count
entire home/apt,13266
private room,11356
shared room,587


2. What is the average price of a listing by room type?

In [10]:
%%sql
SELECT room_type, ROUND(AVG(price),2) average_price
FROM prices p
JOIN room_types rt 
ON p.listing_id = rt.listing_id
GROUP BY 1
ORDER BY 2 DESC

 * sqlite:///palooza.db
Done.


room_type,average_price
entire home/apt,197.17
private room,81.67
shared room,53.65


3. Which borough has the highest average price per month?

In [11]:
%%sql

SELECT borough, ROUND(AVG(price),2) average_price
FROM prices
GROUP BY 1
ORDER BY 2 DESC

 * sqlite:///palooza.db
Done.


borough,average_price
Manhattan,184.0
Brooklyn,121.97
Queens,92.81
Staten Island,86.04
Bronx,79.24


4. How many listings of each room type are in each borough?

In [12]:
%%sql

SELECT borough, room_type, COUNT(room_type) AS room_count
FROM prices p
JOIN room_types rt 
ON p.listing_id = rt.listing_id
GROUP BY 1, 2
ORDER BY 1 ASC, 3 DESC

 * sqlite:///palooza.db
Done.


borough,room_type,room_count
Bronx,private room,403
Bronx,entire home/apt,261
Bronx,shared room,33
Brooklyn,entire home/apt,5367
Brooklyn,private room,4906
Brooklyn,shared room,187
Manhattan,entire home/apt,6170
Manhattan,private room,3901
Manhattan,shared room,251
Queens,private room,2009


5. How many listings in each room type category have a price of over $500 per night?

In [13]:
%%sql

SELECT room_type, COUNT(price) num_listings
FROM prices p
JOIN room_types rt 
ON p.listing_id = rt.listing_id
WHERE price > 500
GROUP BY 1


 * sqlite:///palooza.db
Done.


room_type,num_listings
entire home/apt,395
private room,19
shared room,1


6. What is the distribution of listing prices by neighbourhood?

In [34]:
%%sql

SELECT borough, neighbourhood, 
        MIN(price) AS min_price, 
        MAX(price) AS max_price,
        ROUND(AVG(price),2) AS mean_price
FROM prices
GROUP BY 1, 2
ORDER BY 1
LIMIT 5

 * sqlite:///palooza.db
Done.


borough,neighbourhood,min_price,max_price,mean_price
Bronx,Allerton,35,450,98.93
Bronx,Baychester,53,101,78.6
Bronx,Belmont,24,299,91.88
Bronx,Bronxdale,40,79,53.6
Bronx,Castle Hill,62,86,74.0


7. What is the estimated amount of revenue generated by hosts in each borough?

In [15]:
%%sql 

SELECT borough, SUM(price) / COUNT(price) AS revenue_per_host
FROM prices
GROUP BY 1
ORDER BY 2 DESC

 * sqlite:///palooza.db
Done.


borough,revenue_per_host
Manhattan,184
Brooklyn,121
Queens,92
Staten Island,86
Bronx,79


8. What is the average price per month for listings in each neighborhood?

In [23]:
%%sql 

SELECT neighbourhood, ROUND(AVG(price_per_month),2) AS avg_price
FROM prices
GROUP BY neighbourhood
ORDER BY 2 DESC
LIMIT 5


 * sqlite:///palooza.db
Done.


neighbourhood,avg_price
Sea Gate,24485.42
Tribeca,12066.44
Flatiron District,10404.19
NoHo,10190.33
SoHo,9099.52


9. How many listings have no reviews?


In [17]:
%%sql

SELECT *
FROM reviews
LIMIT 5

 * sqlite:///palooza.db
Done.


index,Unnamed: 0,listing_id,host_name,last_review,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,booked_days_365
0,0,2595,Jennifer,2019-05-21 00:00:00,1,45,0.38,2,355,10
1,1,3831,LisaRoxanne,2019-07-05 00:00:00,1,270,4.64,1,194,171
2,2,5099,Chris,2019-06-22 00:00:00,3,74,0.59,1,129,236
3,3,5178,Shunichi,2019-06-24 00:00:00,2,430,3.47,1,220,145
4,4,5238,Ben,2019-06-09 00:00:00,1,160,1.33,4,188,177


In [26]:
%%sql 

SELECT COUNT(rt.listing_id) num_listings_no_review
FROM room_types rt
LEFT JOIN reviews r 
ON rt.listing_id = r.listing_id
WHERE host_name IS NULL
ORDER BY 1 ASC

 * sqlite:///palooza.db
Done.


num_listings_no_review
8


10. How do the estimated book days correlate with the price of an Airbnb listing in New York City?

In [33]:
%%sql 

SELECT
  (COUNT(p.price * r.booked_days_365) - COUNT(p.price) * AVG(p.price) * AVG(r.booked_days_365)) /
  (COUNT(p.price) * AVG(p.price * p.price) - COUNT(p.price) * AVG(p.price) * AVG(p.price)) AS correlation
FROM prices p
JOIN reviews r
ON p.listing_id = r.listing_id


 * sqlite:///palooza.db
Done.


correlation
-1.4278840527031131


In [ ]:
%%sql